## caching

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache
# chat = ChatOpenAI(temperature=0.1)

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)
# chat = ChatOpenAI(
#     temperature = 0.1,
#     # streaming = True,
#     # callbacks = [StreamingStdOutCallbackHandler()]
# )

# chat.predict("how do you make italian pasta")

## Serialization


In [ ]:
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature = 0.1,
    # streaming = True,
    # callbacks = [StreamingStdOutCallbackHandler()]
)

with get_openai_callback() as usage:
    a = chat.predict("what is the recipe for soju? explain briefly.")
    b = chat.predict("what is the recipe for bread? explain briefly.")
    print(a,b,"\n")
    # print(usage)
    print(usage.total_cost)


## save model

In [3]:
from langchain.llms.openai import OpenAI

# chat = OpenAI(
#     temperature=0.1,
#     max_tokens=450,
#     model="gpt-3.5-turbo-16k"
# )
# chat.save("model.json")

from langchain.llms.loading import load_llm

chat = load_llm("model.json")
chat

c:\Users\jspar\StudioProjects\nomad\fullstack_GPT\env\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\jspar\StudioProjects\nomad\fullstack_GPT\env\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo-16k', model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})

## Memory  
### conversation buffer memory

In [4]:
from langchain.memory import ConversationBufferMemory

# memory = ConversationBufferMemory(return_messages=True) 챗model 위함
memory = ConversationBufferMemory()

memory.save_context({"input": "hi"}, {"output":"how are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'), AIMessage(content='how are you?')]}

### conversation buffer window memory

In [7]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages= True,
    k=4
)

def add_message(input, output):
    memory.save_context({"input":input},{"output": output})
    
add_message(1,1)
add_message(2,2)
add_message(3,3)
add_message(4,4)
add_message(5,5)

memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

### conversation summary (buffer) memory

In [10]:
from langchain.memory import ConversationSummaryMemory,ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input":input},{"output": output})
    
def get_history():
    return memory.load_memory_variables({})

add_message("hi, i'm junsoo, i live in south korea", "wow that so cooool")
add_message("south korea is so pretty ", "I'm dying to go to south korea")
get_history()


{'history': 'The human introduces themselves as Junsoo from South Korea. The AI responds with enthusiasm, saying that it is cool and expresses a desire to visit South Korea because it is so pretty.'}

In [16]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    return_messages=True,
)
add_message("hi, i'm junsoo, i live in south korea", "wow that so cooool")
add_message("south korea is so pretty ", "I'm dying to go to south korea")
get_history()
add_message("why don't we hang out later in my place", "cool~ where is your place in")
get_history()

{'history': [SystemMessage(content='The human introduces themselves as Junsoo from South Korea. The AI expresses excitement and finds it cool. The human mentions that South Korea is beautiful.'),
  AIMessage(content="I'm dying to go to south korea"),
  HumanMessage(content="why don't we hang out later in my place"),
  AIMessage(content='cool~ where is your place in')]}

## conversationKGMemory

In [19]:
from langchain.memory import ConversationKGMemory

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)

add_message("hi, i'm junsoo, i live in south korea", "wow that so cooool")
add_message("south korea is so pretty ", "I'm dying to go to south korea")

add_message("why don't we hang out later in my place", "cool~ where is your place in")
memory.load_memory_variables({"inputs": "who is junsoo"})

{'history': [SystemMessage(content='On junsoo: junsoo is human.')]}

## Memory on LLMChain

In [21]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    memory_key="chat_history"
)

template = """
    You are a helpful AI talking to a human.
    
    {chat_history}
    Human:{question}
    You:
"""


chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt= PromptTemplate.from_template(template),
    verbose=True
)

chain.predict(question="My name is nico")







> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    
    Human:My name is nico
    You:


> Finished chain.


'Hello Nico! How can I assist you today?'

In [22]:
chain.predict(question="I lived in seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human: My name is nico
AI: Hello Nico! How can I assist you today?
    Human:I lived in seoul
    You:


> Finished chain.


"That's great! Seoul is a vibrant and bustling city. How can I assist you today, Nico?"

In [23]:
chain.predict(question= "what is my name")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human: My name is nico
AI: Hello Nico! How can I assist you today?
Human: I lived in seoul
AI: That's great! Seoul is a vibrant and bustling city. How can I assist you today, Nico?
    Human:what is my name
    You:


> Finished chain.


'Your name is Nico.'

## Chat Based Memory

In [ ]:
from langchain.prompts import PromptTemplate,ChatPromptTemplate,MessagesPlaceholder

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    return_messages=True
    
)

prompt = ChatPromptTemplate.from_messages([
    ("system","you are a helpful Ai talking a human"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])


chain.predict(question="My name is nico")


## LCEL Based Memory

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

def load_memory(input):
    print(input)
    return memory.load_memory_variables({})["history"],

chain = RunnablePassthrough.assign(history=load_memory) |prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question,},{"output":result.content})
    print(result)

